In [1]:
import pandas as pd

In [8]:
data = pd.read_csv('ensemble.csv')

In [9]:
data

,index
0,"""mrc-1-000653"": ""사락사라"""
1,"""mrc-1-001113"": ""냉전"""
2,"""mrc-0-002191"": ""대통령인 빌헬름 미클라스"""
3,"""mrc-0-003951"": ""뉴질랜드"""
4,"""mrc-1-001272"": ""프랑스군"""
...,...
595,"""mrc-0-002989"": ""재단 X"""
596,"""mrc-0-001804"": ""사망한 사망자의 유가족"""
597,"""mrc-0-003411"": ""병역 문제"""
598,"""mrc-0-003436"": ""주노 해변"""


In [17]:

from collections import defaultdict

json_dump_data = defaultdict(set)
for i in range(len(data)):
    temp = data['index'][i].split(': ')
    json_dump_data[temp[0][1:len(temp[0])-1]] = temp[1][1:len(temp[1])-1]
#     print(data['index'][i])

In [34]:
from datasets import (
    Dataset,
    load_from_disk,
    concatenate_datasets,
)

org_dataset = load_from_disk("/opt/ml/input/data/dummy_dataset")
full_ds = concatenate_datasets(
    [
        org_dataset["train"].flatten_indices(),
        org_dataset["validation"].flatten_indices(),
    ]
) # train dev 를 합친 4192 개 질문에 대해 모두 테스트
print("*"*40, "query dataset", "*"*40)
print(full_ds)


**************************************** query dataset ****************************************
Dataset({
    features: ['answers', 'context', 'id', 'question', 'title'],
    num_rows: 220
})


In [39]:
cout = 0

for i in range(len(full_ds)):
    if full_ds[i]['id'] in json_dump_data.keys():
        json_dump_data[full_ds[i]['id']] = full_ds[i]['answers']['text'][0]
        cout += 1
        
cout

24

In [40]:
json_dump_data

defaultdict(set,
            {'mrc-1-000653': '사락사라',
             'mrc-1-001113': '냉전',
             'mrc-0-002191': '대통령인 빌헬름 미클라스',
             'mrc-0-003951': '뉴질랜드',
             'mrc-1-001272': '프랑스군',
             'mrc-1-000993': '뒷다리',
             'mrc-0-005021': '민주주의',
             'mrc-1-000163': '대규모의 수증기 폭발',
             'mrc-0-001283': '순조 11년(1811)',
             'mrc-0-004543': '고전도성 철',
             'mrc-0-000439': '점쟁이',
             'mrc-0-002895': '칼라치 전방 약250km 지점',
             'mrc-0-000535': '롭 포드',
             'mrc-1-001724': '노스햄프턴 교회',
             'mrc-0-000901': '아루',
             'mrc-0-001606': '석조 궁륭',
             'mrc-0-000266': '국민',
             'mrc-0-001326': '‘자유’',
             'mrc-0-000032': '국방부 보고서',
             'mrc-0-005215': '입석대(立石臺)',
             'mrc-0-005407': '숙의 정씨',
             'mrc-0-003683': '국가인권위원회',
             'mrc-0-003644': '이탈리아',
             'mrc-0-002835': '망자',
             'mrc-0-000049': '왜냐하면',
             '

In [41]:
def write_json_file(filename, data):
    with open(filename, "w+") as writer:
        try:
            writer.write(json.dumps(data, indent=4, ensure_ascii=False) + "\n")
        except ValueError as e:
            print('Writing failed! Error: {}'.format(e))
            return None

In [42]:
import os, json

In [43]:
json_write_dir = os.path.join('/opt/ml/code/outputs/test_dataset/koelectra-pororo-top10', 'predictions_pp.json')
write_json_file(json_write_dir, json_dump_data)